-------------

Relative Positional Encoding Example

In [1]:
from torch import nn
import torch

# Initialize the d model
d_model = 100

# Give the max position
max_relative_position = 150

# Create a lookup table (if it were inside a model we must initialize as weights)
embeddings_table = nn.Parameter(torch.Tensor(max_relative_position * 2 + 1, d_model))

# Initialize the value of the lookup table with xavier uniform
nn.init.xavier_uniform_(embeddings_table)

c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Parameter containing:
tensor([[-0.0130, -0.0605, -0.0268,  ..., -0.0494, -0.0109,  0.0752],
        [-0.0863,  0.1188, -0.0339,  ..., -0.0840, -0.1023, -0.0929],
        [-0.0645,  0.1090, -0.0522,  ..., -0.1137,  0.0795, -0.0604],
        ...,
        [ 0.0685,  0.1095, -0.0067,  ...,  0.0524, -0.0245, -0.0998],
        [ 0.0105, -0.0902, -0.0232,  ...,  0.0082,  0.1142,  0.0036],
        [-0.0877,  0.0690, -0.0744,  ..., -0.0061,  0.0501,  0.0011]],
       requires_grad=True)

In [2]:
embeddings_table.size() # size = (number of positions, embedding dimension)

torch.Size([301, 100])

In [3]:
# Initialize a range with same size as the query sequence length (200)
range_vec_q = torch.arange(200)

# Initialize a range with same size as the key sequence length (200)
range_vec_k = torch.arange(200)

# Calculate the distance between the positions
distance_mat = range_vec_k[None, :] - range_vec_q[:, None]

# Clip the distances between the -max distance and max distance
distance_mat_clipped = torch.clamp(distance_mat, -150, 150)

# Add max distance to the clipped distance in order to obtain positive indices
final_mat = distance_mat_clipped + 150

# Transform final matrix to Long in order to map the indices to their vectors
final_mat = torch.LongTensor(final_mat)

# Determine the positional embeddings
embeddings = embeddings_table[final_mat]

In [4]:
embeddings.size() # The embedding is of dimension [query sequence length, keys sequence length, embedding dim]

torch.Size([200, 200, 100])

**Note since we are using the following formula from [relative_position](https://arxiv.org/pdf/1803.02155v2)**:

$$
E = \frac{Attention + Additional\_attention}{\sqrt{d\_model}}
$$

Where $Additional\_attention = Linear(Q) \times position\_embeddings$

And that we don't need to determine a first relative positional embedding to add to the linear transformation of the values because we are making machine translation. Then the key sequence length that we used in `range_vec_k` can be taken as the query sequence length. But for more comprehension of the process we will maintain the name.

In [11]:
# Initialize a query with size = (batch = 4, sequence, d_model)
query = torch.randn((4, 200, 100))

# Calculate the additional attention
add_attention = query.transpose(0, 1).matmul(embeddings.transpose(1, 2)).transpose(0, 1)

Notice that we transpose the batch dimension with the sequence dimension of the query matrix and we also transposed the dimension of the key sequence with the embedding dimension in order to make matrix multiplication between the query and the position embeddings. After we replaced the batch dimension on its position.

In [12]:
add_attention.size() 

torch.Size([4, 200, 200])

The result will be then added to the original attention and all together divided by $\sqrt{d\_model}$.